# Importing libraries

In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from datetime import datetime
import time
import re
import random
import string

# Accessing to the website

In [2]:
driver = webdriver.Firefox()
driver.get('https://seguro-moto.rastreator.com/datos-comparativa/Q1')
time.sleep(5)
button_cookies = driver.find_element_by_xpath("/html/body/div[4]/div/div[2]/button[2]")
time.sleep(3)
button_cookies.click()

# Scrapping

In [9]:
bbdd_moto = []
count = 1

#list_marcas = driver.find_element_by_xpath('/html/body/app-root/app-single-question/div[3]/div/div/div/div/div/cj-make/div/div/div/ul[1]').text.split("\n")
list_marcas = ['PIAGGIO']


for marca in list_marcas:  
    print(f'_________________________\n{marca} ({count}/{len(list_marcas)})\n--------------------------')
    path_marca = f'//*[@id="li_top_{marca}"]'
    button_marca = driver.find_element_by_xpath(path_marca)
    button_marca.click()
    time.sleep(2)
    list_usados = driver.find_element_by_xpath('/html/body/app-root/app-single-question/div[3]/div/div/div/div/div/cj-model/div/div[2]/div/ul[1]').text.split("\n")
    counter = 1
    for modelo in [list_usados]:
        print(f'- {modelo}, {counter}/{len(list_usados)}')
        counter+=1
        button_modelo = driver.find_element_by_xpath(f'//*[@id="li_{modelo}"]')
        button_modelo.click()
        time.sleep(2)
        page = driver.current_url[-2:]
        if page == 'Q3':
            lista_cc = driver.find_element_by_xpath('//*[@id="card"]').text.split('\n')
            for j in range(3,len(lista_cc)-1):
                button_cc = driver.find_element_by_xpath(f'/html/body/app-root/app-single-question/div[3]/div/div/div/div/div/cj-process-cubic-capacity/div/div[{j}]')
                button_cc.click()
                cilindr = int(lista_cc[j].split(' ')[0])
                time.sleep(2)
                page = driver.current_url[-2:]
                if page == 'Q4' and cilindr <= 125 and cilindr > 0:
                    modelo_lanz = driver.find_element_by_xpath('//*[@id="questionContainer"]').text.split('\n')
                    k = 1
                    pg = 1
                    while k < len(modelo_lanz) and int(modelo_lanz[k+1][-4:]) >= 2020 and modelo_lanz[k+1][-4:] != '9 CV':
                        #bbdd_moto.append([marca, modelo, lista_cc[j], modelo_lanz[k], modelo_lanz[k+1]])
                        lista_it = [marca, modelo, lista_cc[j], modelo_lanz[k], modelo_lanz[k+1].split(' | ')[1], int(modelo_lanz[k+1][-4:])]
                        time.sleep(2)
                        driver.find_element_by_xpath(f'/html/body/app-root/app-single-question/div[3]/div/div/div/div/div/cj-version/div/div[{pg}]/label/div/span').click()
                        time.sleep(2)
                        driver.find_element_by_xpath("/html/body/app-root/app-single-question/div[3]/div/div/div/div/div/cj-already-bought/div/div[1]/label/div/span").click()
                        time.sleep(2)
                            
                        for year_old in [0]:
                            year_old_new = int(lista_it[5])+year_old
                            if year_old_new > 2021:
                                pass
                            else:
                                for year_mat in [0,2]:
                                    year_mat_new = int(lista_it[5])+year_mat
                                    if year_mat_new > 2021:
                                        pass
                                    else:
                                        if year_old>year_mat:
                                            pass
                                        else:
                                            select_monthOld = Select(driver.find_element_by_id('monthOld'))
                                            select_monthOld.select_by_visible_text('Junio')
                                            
                                            select_buyMonth = Select(driver.find_element_by_id('buyMonth'))
                                            select_buyMonth.select_by_visible_text('Junio')
                                            
                                            select_yearOld = Select(driver.find_element_by_id('yearOld'))
                                            select_yearOld.select_by_visible_text(f'{year_old_new}')
                                            
                                            select_buyYear = Select(driver.find_element_by_id('buyYear'))
                                            
                                            select_buyYear.select_by_visible_text(f'{year_mat_new}')
                                            lista_year = lista_it+[year_mat_new]
                                            
                                            time.sleep(2)
                                            
                                            driver.find_element_by_xpath('//*[@id="next"]').click()
                                            time.sleep(2)
                                            driver.find_element_by_xpath("/html/body/app-root/app-single-question/div[3]/div/div/div/div/div/cj-process-bike-usage/div/div[2]/label/div/span").click()
                                            time.sleep(2)
                                            driver.find_element_by_xpath("/html/body/app-root/app-single-question/div[3]/div/div/div/div/div/cj-process-passenger-type/div/div[3]/label/div/span").click()
                                            time.sleep(2)
                                            driver.find_element_by_xpath(f'/html/body/app-root/app-single-question/div[3]/div/div/div/div/div/cj-process-parking-type/div/div[2]/label/div').click()
                                            time.sleep(2)
                                            kms = driver.find_element_by_xpath('//*[@id="questionContainer"]').text.split('\n')
                                            #for km in range(2,len(kms),3):
                                            for km in [6]:
                                                #lista_km = []
                                                #lista_km = lista_year+[driver.find_element_by_xpath(f'/html/body/app-root/app-single-question/div[3]/div/div/div/div/div/cj-process-annual-km/div/div[{km}]/label/div/span').text]
                                                driver.find_element_by_xpath(f'/html/body/app-root/app-single-question/div[3]/div/div/div/div/div/cj-process-annual-km/div/div[{km}]/label/div/span').click()
                                                time.sleep(2)
                                                                                     
                                                for year_birth in ['2002', '1999', '1995']:
                                                    driver.refresh()
                                                    time.sleep(3)
                                                    
                                                    input_day = driver.find_element_by_xpath('//*[@id="day"]')
                                                    input_day.clear()
                                                    input_day.send_keys('06')
                                                    time.sleep(2)
                                                    
                                                    select_month = Select(driver.find_element_by_xpath('//*[@id="month"]'))
                                                    select_month.select_by_visible_text('Junio')
                                                    time.sleep(2)
                                                    
                                                    input_year = driver.find_element_by_xpath('//*[@id="year"]')
                                                    input_year.clear()
                                                    input_year.send_keys(year_birth)
                                                    time.sleep(2)
                                                    lista_birth = []
                                                    lista_birth = lista_year + [2021-int(year_birth)]
                                                    #lista_birth = lista_km + [2021-int(year_birth)]
                                                    driver.find_element_by_xpath('//*[@id="next"]').click()
                                                    time.sleep(2)
                                                    driver.find_element_by_xpath("/html/body/app-root/app-single-question/div[3]/div/div/div/div/div/cj-process-birth-place/div/div/div[1]/div/label/div/div/img").click()
                                                    time.sleep(2)
                                                    #for gen in range(1,3):
                                                    for gen in [1]:
                                                        genero = driver.find_element_by_xpath(f"/html/body/app-root/app-single-question/div[3]/div/div/div/div/div/cj-process-main-driver-data/div/div[2]/div[{gen}]/div/label/div/p")
                                                        #lista_gen = []
                                                        #lista_gen = lista_birth + [genero.text]
                                                        genero.click()
                                                        time.sleep(2)
                                                        try:
                                                            driver.find_element_by_xpath('/html/body/app-root/app-single-question/div[3]/div/div/div/div/div/cj-process-main-driver-data/div/div[4]/div/div[1]/label/div/span').click()
                                                        except:
                                                            pass
                                                        time.sleep(2)
                                                        driver.find_element_by_xpath('/html/body/app-root/app-single-question/div[3]/div/div/div/div/div/cj-process-profession/div/div[18]/label/div/span').click()
                                                        time.sleep(2)
                                                        
                                                        #for cp in range(1,53):
                                                        for cp in [8,28]:
                                                            if cp < 10:
                                                                postcode = f'0{cp}001'
                                                            else:
                                                                postcode = f'{cp}001'
                                                            input_cp = driver.find_element_by_id("postalCode")
                                                            input_cp.clear()
                                                            input_cp.send_keys(postcode)
                                                            lista_cp = []
                                                            lista_cp = lista_birth + [str(postcode)]
                                                            time.sleep(2)
                                                        
                                                            driver.find_element_by_xpath('//*[@id="next"]').click()
                                                            time.sleep(2)
                                                            driver.find_element_by_xpath('/html/body/app-root/app-single-question/div[3]/div/div/div/div/div/app-bike-driver-childs/div/div/div/div[2]/label/div/span').click()
                                                            time.sleep(2)
                                                            
                                                            lista_c = driver.find_element_by_xpath('//*[@id="questionContainer"]').text.split('\n')[2:]
                                                            for carn in range(2,len(lista_c)+2):
                                                                
                                                                carnet = driver.find_element_by_xpath(f'/html/body/app-root/app-single-question/div[3]/div/div/div/div/div/cj-process-licence-type/div/div[{carn}]/label/div/span')
                                                                lista_carnet = []
                                                                lista_carnet = lista_cp + [carnet.text]
                                                                
                                                                    
                                                                if carnet.text == 'B (carné de coche)':
                                                                    pass
                                                                
                                                                else:
                                                                    if year_birth == '2002':
                                                                        obt_carnet = ['2021']
                                                                    elif year_birth == '1999':
                                                                        obt_carnet = ['2021','2018']
                                                                    else:
                                                                        obt_carnet = ['2021','2018','2014']    
                                                                    
                                                                    carnet.click()
                                                                    time.sleep(2)
                                                                    
                                                                    for year_carn in obt_carnet:
                                                                        
                                                                        driver.refresh()
                                                                        time.sleep(5)
                                                                        
                                                                        select_month = Select(driver.find_element_by_xpath('//*[@id="year"]'))
                                                                        select_month.select_by_visible_text(year_carn)
                                                                        time.sleep(2)
                                                                        
                                                                        select_month = Select(driver.find_element_by_xpath('//*[@id="month"]'))
                                                                        select_month.select_by_visible_text('Julio')
                                                                        time.sleep(2)
                                                                        
                                                                        lista_obt = []
                                                                        lista_obt = lista_carnet + [int(year_carn)-int(year_birth)]
                                                                        page_input = driver.current_url
                                                                        
                                                                        driver.find_element_by_xpath('//*[@id="next"]').click()
                                                                        time.sleep(2)
                                                                        driver.refresh()
                                                                        time.sleep(3)
                                                                        
                                                                        driver.find_element_by_xpath('/html/body/app-root/app-single-question/div[3]/div/div/div/div/div/cj-process-licence-place/div/div/div[1]/div/label/div/p').click()
                                                                        time.sleep(2)
                                                                        try:
                                                                            driver.find_element_by_xpath('/html/body/app-root/app-single-question/div[3]/div/div/div/div/div/app-last-bike/div/div/div/div[1]/label/div/span').click()
                                                                        except:
                                                                            pass
                                                                        time.sleep(2)
                                                                        driver.find_element_by_xpath('/html/body/app-root/app-single-question/div[3]/div/div/div/div/div/cj-process-who-policy-holder/div/div[1]/label/div/span').click()
                                                                        time.sleep(2)
                                                                        driver.find_element_by_xpath('/html/body/app-root/app-single-question/div[3]/div/div/div/div/div/app-defaulter/div/div[2]/label/div/span').click()
                                                                        driver.refresh()
                                                                        time.sleep(5)
                                                        
                                                                        input_dni = driver.find_element_by_xpath('//*[@id="txtDniNumber"]')
                                                                        input_dni.clear()
                                                                        input_dni.send_keys('73115063H')
                                                                        time.sleep(2)
                                                                        driver.find_element_by_xpath('//*[@id="next"]').click()
                                                                        
                                                                        time.sleep(2)
                                                                        driver.find_element_by_xpath('/html/body/app-root/app-single-question/div[3]/div/div/div/div/div/cj-process-insurer-name/div/div[1]/div/div[2]/label/div/span/font/font').click()
                                                                        time.sleep(2)
                                                                        driver.find_element_by_xpath('/html/body/app-root/app-single-question/div[3]/div/div/div/div/div/app-has-claim/div/div[2]/div/div[2]/label/div/span').click()
                                                                        time.sleep(2)
                                                                        driver.find_element_by_xpath('/html/body/app-root/app-single-question/div[3]/div/div/div/div/div/cj-process-previous-fine/div/div[2]/div/div[2]/label/div/span').click()
                                                                        time.sleep(2)
                                                                        select_month = Select(driver.find_element_by_xpath('//*[@id="day"]'))
                                                                        select_month.select_by_visible_text('1')
                                                                        time.sleep(2)
                                                                        select_month = Select(driver.find_element_by_xpath('//*[@id="month"]'))
                                                                        select_month.select_by_visible_text('Septiembre')
                                                                        time.sleep(2)
                                                                        select_month = Select(driver.find_element_by_xpath('//*[@id="year"]'))
                                                                        select_month.select_by_visible_text('2021')
                                                                        time.sleep(2)
                                                                        driver.find_element_by_xpath('//*[@id="next"]').click()
                                                                        time.sleep(2)
                                                                        driver.find_element_by_xpath('/html/body/app-root/app-single-question/div[3]/div/div/div/div/div/cj-process-kind-of-insurance/div/div[5]/label/div/span').click()
                                                                        driver.refresh()
                                                                        time.sleep(5)
                                                        
                                                                        input_mail = driver.find_element_by_xpath('//*[@id="email"]')
                                                                        input_mail.clear()
                                                                        input_mail.send_keys('xabilopez8@hotmail.com')
                                                                        time.sleep(2)
                                                                        input_phone = driver.find_element_by_xpath('//*[@id="phone"]')
                                                                        input_phone.clear()
                                                                        input_phone.send_keys('606697311')
                                                                        time.sleep(2)
                                                                        driver.find_element_by_xpath("/html/body/app-root/app-single-question/div[3]/div/div/div/div/div/cj-process-email-info/div/div[3]/label/div/span[1]").click()
                                                                        
                                                                        rastreator_last = driver.current_url
                                                                        
                                                                        driver.find_element_by_xpath('/html/body/app-root/app-single-question/div[3]/div/div/div/div/div/cj-process-email-info/app-nav-option/a[2]/font/font').click()
                                                                                                                                   
                                                                        time.sleep(30)
                                                                        while driver.current_url == 'https://presupuesto-moto.rastreator.com/resultados-comparativa/isWaitPageCounterInserted':
                                                                            time.sleep(2)
                                                                            driver.refresh()
                                                                        time.sleep(2)
                                                                        
                                                                        try:
                                                                            insur = driver.find_element_by_xpath('/html/body/app-root/div[1]/app-quote-page/div/div/div[1]/filter/div/div').text.split('\n')
                                                                            for ins in range(0,len(insur),3):
                                                                                insurance = insur[ins]
                                                                                precio = str(insur[ins+2].split('desde')[1])
                                                                                lista_ins = [] 
                                                                                lista_ins = lista_obt + [insurance] + [precio]
                                                                                bbdd_moto.append(lista_ins)
                                                                                #df = pd.DataFrame(bbdd_moto, columns = ['Marca','Modelo','Cilindrada','Versión','Potencia','Año lanzamiento','Año matriculación','Año compra','KM esperados','Edad','Género','Código postal','Tipo carnet','Años carnet','Tipo seguro','Precio mínimo'])
                                                                                df = pd.DataFrame(bbdd_moto, columns = ['Marca','Modelo','Cilindrada','Versión','Potencia','Año lanzamiento','Año matriculación','Edad actual','Código postal','Tipo carnet','Edad carnet','Tipo seguro','Precio mínimo'])
                                                                                df.to_csv('../data/dataset_piaggio.csv', index=False)
                                                                                print(lista_ins)
                                                                        except:
                                                                            pass
                                                                        
                                                                        while driver.current_url[:-2] != 'https://seguro-moto.rastreator.com/datos-comparativa/Q':
                                                                            driver.back()
                                                                            #time.sleep(0.5)
                                                                        driver.refresh()
                                                                        time.sleep(3)
                                                                        while driver.current_url[:-2] != 'https://seguro-moto.rastreator.com/datos-comparativa/Q':
                                                                            driver.back()
                                                                        driver.refresh()
                                                                        time.sleep(10)
                                                                        while page_input != driver.current_url:
                                                                            driver.back()
                                                                            time.sleep(3)
                                                                        driver.refresh()
                                                                        time.sleep(5)
                                                                    
                                                                    driver.back()
                                                                    time.sleep(2)
                                                            driver.back()
                                                            time.sleep(2)
                                                            driver.back()
                                                            time.sleep(2)
                           
                        
                                                        driver.back()
                                                        time.sleep(2)
                                
                        
                                                        driver.back()
                                                        time.sleep(2)
                                                    driver.back()
                                                    time.sleep(2)
                                                    driver.back()
                                                    time.sleep(2)
                                                driver.back()
                                                time.sleep(2)
                                            driver.back()
                                            time.sleep(2)
                                            driver.back()
                                            time.sleep(2)
                                            driver.back()
                                            time.sleep(2)
                                            driver.back()
                                            time.sleep(2)
                                            
                        driver.back()
                        time.sleep(2)
                        driver.back()
                        time.sleep(2)
                        pg += 1
                        k+=2
                        
                else:
                    pass
                driver.back()
                time.sleep(2)  
        else:
            pass
        driver.back()
        time.sleep(2)
    driver.back()
    time.sleep(2)
    count += 1
    
    

_________________________
PIAGGIO (1/1)
--------------------------
- VESPA 50, 1/18
['PIAGGIO', 'VESPA 50', '50 cc', 'VESPA 50 SPRINT RACING SIXTIES', '3 CV', 2020, 2020, 19, '08001', 'A1 (para conducir motos de hasta 125cc)', 19, 'Terceros sin asistencia en viaje', '395€']
['PIAGGIO', 'VESPA 50', '50 cc', 'VESPA 50 SPRINT RACING SIXTIES', '3 CV', 2020, 2020, 19, '08001', 'A1 (para conducir motos de hasta 125cc)', 19, 'Terceros con asistencia en viaje', '411€']
['PIAGGIO', 'VESPA 50', '50 cc', 'VESPA 50 SPRINT RACING SIXTIES', '3 CV', 2020, 2020, 19, '08001', 'A1 (para conducir motos de hasta 125cc)', 19, 'Terceros ampliado', '1.176€']
['PIAGGIO', 'VESPA 50', '50 cc', 'VESPA 50 SPRINT RACING SIXTIES', '3 CV', 2020, 2020, 19, '08001', 'A2 (para motos de hasta 35kw y 0,2 kw/kg o motos limitadas)', 19, 'Terceros sin asistencia en viaje', '332€']
['PIAGGIO', 'VESPA 50', '50 cc', 'VESPA 50 SPRINT RACING SIXTIES', '3 CV', 2020, 2020, 19, '08001', 'A2 (para motos de hasta 35kw y 0,2 kw/kg o m

['PIAGGIO', 'VESPA 50', '50 cc', 'VESPA 50 SPRINT RACING SIXTIES', '3 CV', 2020, 2020, 22, '28001', 'Licencia de ciclomotor', 22, 'Terceros sin asistencia en viaje', '284€']
['PIAGGIO', 'VESPA 50', '50 cc', 'VESPA 50 SPRINT RACING SIXTIES', '3 CV', 2020, 2020, 22, '28001', 'Licencia de ciclomotor', 22, 'Terceros con asistencia en viaje', '300€']
['PIAGGIO', 'VESPA 50', '50 cc', 'VESPA 50 SPRINT RACING SIXTIES', '3 CV', 2020, 2020, 22, '28001', 'Licencia de ciclomotor', 19, 'Terceros sin asistencia en viaje', '193€']
['PIAGGIO', 'VESPA 50', '50 cc', 'VESPA 50 SPRINT RACING SIXTIES', '3 CV', 2020, 2020, 22, '28001', 'Licencia de ciclomotor', 19, 'Terceros con asistencia en viaje', '210€']
['PIAGGIO', 'VESPA 50', '50 cc', 'VESPA 50 SPRINT RACING SIXTIES', '3 CV', 2020, 2020, 26, '08001', 'A1 (para conducir motos de hasta 125cc)', 26, 'Terceros sin asistencia en viaje', '238€']
['PIAGGIO', 'VESPA 50', '50 cc', 'VESPA 50 SPRINT RACING SIXTIES', '3 CV', 2020, 2020, 26, '08001', 'A1 (para con

['PIAGGIO', 'VESPA 50', '50 cc', 'VESPA 50 SPRINT RACING SIXTIES', '3 CV', 2020, 2020, 26, '28001', 'Licencia de ciclomotor', 26, 'Terceros sin asistencia en viaje', '234€']
['PIAGGIO', 'VESPA 50', '50 cc', 'VESPA 50 SPRINT RACING SIXTIES', '3 CV', 2020, 2020, 26, '28001', 'Licencia de ciclomotor', 26, 'Terceros con asistencia en viaje', '250€']
['PIAGGIO', 'VESPA 50', '50 cc', 'VESPA 50 SPRINT RACING SIXTIES', '3 CV', 2020, 2020, 26, '28001', 'Licencia de ciclomotor', 23, 'Terceros sin asistencia en viaje', '168€']
['PIAGGIO', 'VESPA 50', '50 cc', 'VESPA 50 SPRINT RACING SIXTIES', '3 CV', 2020, 2020, 26, '28001', 'Licencia de ciclomotor', 23, 'Terceros con asistencia en viaje', '185€']
['PIAGGIO', 'VESPA 50', '50 cc', 'VESPA 50 SPRINT RACING SIXTIES', '3 CV', 2020, 2020, 26, '28001', 'Licencia de ciclomotor', 19, 'Terceros sin asistencia en viaje', '129€']
['PIAGGIO', 'VESPA 50', '50 cc', 'VESPA 50 SPRINT RACING SIXTIES', '3 CV', 2020, 2020, 26, '28001', 'Licencia de ciclomotor', 19, 

['PIAGGIO', 'VESPA 125', '125 cc', 'VESPA 125 SPRINT RACING SIXTIES', '11 CV', 2020, 2020, 22, '28001', 'A1 (para conducir motos de hasta 125cc)', 19, 'Terceros sin asistencia en viaje', '149€']
['PIAGGIO', 'VESPA 125', '125 cc', 'VESPA 125 SPRINT RACING SIXTIES', '11 CV', 2020, 2020, 22, '28001', 'A1 (para conducir motos de hasta 125cc)', 19, 'Terceros con asistencia en viaje', '166€']
['PIAGGIO', 'VESPA 125', '125 cc', 'VESPA 125 SPRINT RACING SIXTIES', '11 CV', 2020, 2020, 22, '28001', 'A1 (para conducir motos de hasta 125cc)', 19, 'Terceros ampliado', '283€']
['PIAGGIO', 'VESPA 125', '125 cc', 'VESPA 125 SPRINT RACING SIXTIES', '11 CV', 2020, 2020, 22, '28001', 'A1 (para conducir motos de hasta 125cc)', 19, 'Todo riesgo con franquicia', '495€']
['PIAGGIO', 'VESPA 125', '125 cc', 'VESPA 125 SPRINT RACING SIXTIES', '11 CV', 2020, 2020, 22, '28001', 'A2 (para motos de hasta 35kw y 0,2 kw/kg o motos limitadas)', 22, 'Terceros sin asistencia en viaje', '189€']
['PIAGGIO', 'VESPA 125', '

['PIAGGIO', 'VESPA 125', '125 cc', 'VESPA 125 SPRINT RACING SIXTIES', '11 CV', 2020, 2020, 26, '28001', 'A1 (para conducir motos de hasta 125cc)', 19, 'Terceros sin asistencia en viaje', '95€']
['PIAGGIO', 'VESPA 125', '125 cc', 'VESPA 125 SPRINT RACING SIXTIES', '11 CV', 2020, 2020, 26, '28001', 'A1 (para conducir motos de hasta 125cc)', 19, 'Terceros con asistencia en viaje', '112€']
['PIAGGIO', 'VESPA 125', '125 cc', 'VESPA 125 SPRINT RACING SIXTIES', '11 CV', 2020, 2020, 26, '28001', 'A1 (para conducir motos de hasta 125cc)', 19, 'Terceros ampliado', '182€']
['PIAGGIO', 'VESPA 125', '125 cc', 'VESPA 125 SPRINT RACING SIXTIES', '11 CV', 2020, 2020, 26, '28001', 'A1 (para conducir motos de hasta 125cc)', 19, 'Todo riesgo con franquicia', '312€']
['PIAGGIO', 'VESPA 125', '125 cc', 'VESPA 125 SPRINT RACING SIXTIES', '11 CV', 2020, 2020, 26, '28001', 'A2 (para motos de hasta 35kw y 0,2 kw/kg o motos limitadas)', 26, 'Terceros sin asistencia en viaje', '152€']
['PIAGGIO', 'VESPA 125', '1

['PIAGGIO', 'MEDLEY', '125 cc', 'MEDLEY 125 I-GET', '15 CV', 2020, 2020, 22, '08001', 'A2 (para motos de hasta 35kw y 0,2 kw/kg o motos limitadas)', 19, 'Terceros sin asistencia en viaje', '163€']
['PIAGGIO', 'MEDLEY', '125 cc', 'MEDLEY 125 I-GET', '15 CV', 2020, 2020, 22, '08001', 'A2 (para motos de hasta 35kw y 0,2 kw/kg o motos limitadas)', 19, 'Terceros con asistencia en viaje', '180€']
['PIAGGIO', 'MEDLEY', '125 cc', 'MEDLEY 125 I-GET', '15 CV', 2020, 2020, 22, '08001', 'A2 (para motos de hasta 35kw y 0,2 kw/kg o motos limitadas)', 19, 'Terceros ampliado', '309€']
['PIAGGIO', 'MEDLEY', '125 cc', 'MEDLEY 125 I-GET', '15 CV', 2020, 2020, 22, '08001', 'A2 (para motos de hasta 35kw y 0,2 kw/kg o motos limitadas)', 19, 'Todo riesgo con franquicia', '564€']
['PIAGGIO', 'MEDLEY', '125 cc', 'MEDLEY 125 I-GET', '15 CV', 2020, 2020, 22, '28001', 'A1 (para conducir motos de hasta 125cc)', 22, 'Terceros sin asistencia en viaje', '210€']
['PIAGGIO', 'MEDLEY', '125 cc', 'MEDLEY 125 I-GET', '15 

['PIAGGIO', 'MEDLEY', '125 cc', 'MEDLEY 125 I-GET', '15 CV', 2020, 2020, 26, '28001', 'A1 (para conducir motos de hasta 125cc)', 19, 'Terceros sin asistencia en viaje', '95€']
['PIAGGIO', 'MEDLEY', '125 cc', 'MEDLEY 125 I-GET', '15 CV', 2020, 2020, 26, '28001', 'A1 (para conducir motos de hasta 125cc)', 19, 'Terceros con asistencia en viaje', '112€']
['PIAGGIO', 'MEDLEY', '125 cc', 'MEDLEY 125 I-GET', '15 CV', 2020, 2020, 26, '28001', 'A1 (para conducir motos de hasta 125cc)', 19, 'Terceros ampliado', '182€']
['PIAGGIO', 'MEDLEY', '125 cc', 'MEDLEY 125 I-GET', '15 CV', 2020, 2020, 26, '28001', 'A1 (para conducir motos de hasta 125cc)', 19, 'Todo riesgo con franquicia', '312€']
['PIAGGIO', 'MEDLEY', '125 cc', 'MEDLEY 125 I-GET', '15 CV', 2020, 2020, 26, '28001', 'A2 (para motos de hasta 35kw y 0,2 kw/kg o motos limitadas)', 23, 'Terceros sin asistencia en viaje', '109€']
['PIAGGIO', 'MEDLEY', '125 cc', 'MEDLEY 125 I-GET', '15 CV', 2020, 2020, 26, '28001', 'A2 (para motos de hasta 35kw y

['PIAGGIO', 'MEDLEY', '125 cc', 'MEDLEY 125 S I-GET', '15 CV', 2020, 2020, 22, '28001', 'A1 (para conducir motos de hasta 125cc)', 19, 'Terceros sin asistencia en viaje', '149€']
['PIAGGIO', 'MEDLEY', '125 cc', 'MEDLEY 125 S I-GET', '15 CV', 2020, 2020, 22, '28001', 'A1 (para conducir motos de hasta 125cc)', 19, 'Terceros con asistencia en viaje', '166€']
['PIAGGIO', 'MEDLEY', '125 cc', 'MEDLEY 125 S I-GET', '15 CV', 2020, 2020, 22, '28001', 'A1 (para conducir motos de hasta 125cc)', 19, 'Terceros ampliado', '283€']
['PIAGGIO', 'MEDLEY', '125 cc', 'MEDLEY 125 S I-GET', '15 CV', 2020, 2020, 22, '28001', 'A1 (para conducir motos de hasta 125cc)', 19, 'Todo riesgo con franquicia', '495€']
['PIAGGIO', 'MEDLEY', '125 cc', 'MEDLEY 125 S I-GET', '15 CV', 2020, 2020, 22, '28001', 'A2 (para motos de hasta 35kw y 0,2 kw/kg o motos limitadas)', 22, 'Terceros sin asistencia en viaje', '189€']
['PIAGGIO', 'MEDLEY', '125 cc', 'MEDLEY 125 S I-GET', '15 CV', 2020, 2020, 22, '28001', 'A2 (para motos de

['PIAGGIO', 'MEDLEY', '125 cc', 'MEDLEY 125 S I-GET', '15 CV', 2020, 2020, 26, '28001', 'A2 (para motos de hasta 35kw y 0,2 kw/kg o motos limitadas)', 26, 'Terceros sin asistencia en viaje', '152€']
['PIAGGIO', 'MEDLEY', '125 cc', 'MEDLEY 125 S I-GET', '15 CV', 2020, 2020, 26, '28001', 'A2 (para motos de hasta 35kw y 0,2 kw/kg o motos limitadas)', 26, 'Terceros con asistencia en viaje', '169€']
['PIAGGIO', 'MEDLEY', '125 cc', 'MEDLEY 125 S I-GET', '15 CV', 2020, 2020, 26, '28001', 'A2 (para motos de hasta 35kw y 0,2 kw/kg o motos limitadas)', 26, 'Terceros ampliado', '277€']
['PIAGGIO', 'MEDLEY', '125 cc', 'MEDLEY 125 S I-GET', '15 CV', 2020, 2020, 26, '28001', 'A2 (para motos de hasta 35kw y 0,2 kw/kg o motos limitadas)', 26, 'Todo riesgo con franquicia', '564€']
['PIAGGIO', 'MEDLEY', '125 cc', 'MEDLEY 125 S I-GET', '15 CV', 2020, 2020, 26, '28001', 'A2 (para motos de hasta 35kw y 0,2 kw/kg o motos limitadas)', 23, 'Terceros sin asistencia en viaje', '109€']
['PIAGGIO', 'MEDLEY', '125